## Yo this notebook is lit

In [1]:
import pandas as pd
import numpy as np
from skimage.io import imshow
from sklearn.model_selection import train_test_split

In [2]:
import keras
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
y_train_labels = [class_labels[int(val)] for val in y_train]
h = 32
w = 32

X_train = X_train/255.0
X_test = X_test/255.0
X_train = X_train - 1/2
X_test = X_test - 1/2

# Takes only the classes we are concerned with 
sub_sample = [1, 4, 9]
X_sub_train = []
y_sub_train = []
for i in range(0,len(y_train)):
    if y_train[i][0] in sub_sample:
        X_sub_train.append(X_train[i])
        if y_train[i][0] == 1 or y_train[i][0] == 9:
            y_sub_train.append([0])
        if y_train[i][0] == 4:
            y_sub_train.append([1])

        
X_sub_test = []
y_sub_test = []
for i in range(0,len(y_test)):
    if y_test[i][0] in sub_sample:
        X_sub_test.append(X_test[i])
        # If it's a truck or automobile
        if y_test[i][0] == 1 or y_test[i][0] == 9:
            y_sub_test.append([0])
        # If it's a deer
        if y_test[i][0] == 4:
            y_sub_test.append([1])

X_sub_train = np.array(X_sub_train)
X_sub_test = np.array(X_sub_test)
y_sub_train = np.array(y_sub_train)
y_sub_test = np.array(y_sub_test)
        
sub_class_labels = ['automobile', 'deer']
y_sub_train_labels = [sub_class_labels[int(val)] for val in y_sub_train]

NUM_CLASSES = len(sub_class_labels)
y_sub_train_ohe = keras.utils.to_categorical(y_sub_train, NUM_CLASSES)
y_sub_test_ohe = keras.utils.to_categorical(y_sub_test, NUM_CLASSES)

Using TensorFlow backend.


In [3]:
#print(X_sub_train)
#print(X_train)

In [4]:
imshow(X_sub_train[0].squeeze())

In [6]:
print(len(X_sub_train))
print(len(y_sub_train))
print(len(X_sub_test))
print(len(y_sub_test))

15000
15000
3000
3000


In [7]:
#X_c = np.concatenate([X_train,X_test], axis=0)
#y_c = np.concatenate([y_train,y_test], axis=0).flatten()
#vectorized_color = np.reshape(X_c,(len(X_c),32*32,3))
#color_pixels = np.reshape(vectorized_color,(len(vectorized_color)*32*32,3))

In [8]:
#bw_pixels = [(0.21*pixel[0])+(0.72*pixel[1])+(0.07*pixel[2]) for pixel in color_pixels]#convert rgb to b/w
#vectorized_bw = np.reshape(bw_pixels,(len(X),32*32))#reconstruct to list in image vectors
#vectorized_bw.dump("data/vectorized_bw")

In [9]:
'''vectorized_bw_all = np.load("data/vectorized_bw")
print(vectorized_bw_all.shape)

#we might need these DF's later
df_bw = pd.DataFrame(data=vectorized_bw_all)
df_bw = df_bw.assign(y=pd.Series(y,name='y'))
#df_bw_sample = df_bw.sample(5000)
X = df_bw.drop('y', axis=1).as_matrix()
y = df_bw.y.as_matrix()
print(X.shape)
print(y.shape)'''
#now, we can do cv or a train test split

'vectorized_bw_all = np.load("data/vectorized_bw")\nprint(vectorized_bw_all.shape)\n\n#we might need these DF\'s later\ndf_bw = pd.DataFrame(data=vectorized_bw_all)\ndf_bw = df_bw.assign(y=pd.Series(y,name=\'y\'))\n#df_bw_sample = df_bw.sample(5000)\nX = df_bw.drop(\'y\', axis=1).as_matrix()\ny = df_bw.y.as_matrix()\nprint(X.shape)\nprint(y.shape)'

In [10]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.layers import average 
from keras.models import Input, Model
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split

In [12]:
datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=5, # used, Int. Degree range for random rotations.
    width_shift_range=0.1, # used, Float (fraction of total width). Range for random horizontal shifts.
    height_shift_range=0.1, # used,  Float (fraction of total height). Range for random vertical shifts.
    shear_range=0., # Float. Shear Intensity (Shear angle in counter-clockwise direction as radians)
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None)

datagen.fit(X_sub_train)

In [49]:
%%time

from keras.layers import average 
from keras.models import Input, Model
from keras.callbacks import EarlyStopping
from keras.regularizers import l2 

l2_lambda = 0.0001

num_ensembles = 3

input_holder = Input(shape=(w, h, 3))

branches = []
for _ in range(num_ensembles):

    conv1 = Conv2D(filters=32,
                   input_shape = (w,h,1),
                   kernel_size=(3,3),
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=l2(l2_lambda),
                   padding='same', 
                   activation='relu')(input_holder)
    
    max1 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv1)

    conv2 = Conv2D(filters=32,
                   kernel_size=(3,3),
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=l2(l2_lambda),
                   padding='same', 
                   activation='relu')(max1)
    
    max2 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv2)


    # add one layer on flattened output
    drop1 = Dropout(0.25)(max2) # add some dropout for regularization after conv layers
    flat1 = Flatten()(drop1)
    dense1 = Dense(128, 
                  activation='relu',
                  kernel_initializer='he_uniform',
                  kernel_regularizer=l2(l2_lambda)
                )(flat1)
    drop2 = Dropout(0.5)(dense1) # add some dropout for regularization, again!
    dense2 = Dense(NUM_CLASSES, 
                  activation='sigmoid', 
                  kernel_initializer='glorot_uniform',
                  kernel_regularizer=l2(l2_lambda)
                 )(drop2)
    
    # now add this branch onto the master list
    branches.append(dense2)

# that's it, we just need to average the results
ave = average(branches)

# here is the secret sauce for setting the network using the 
#   Model API:
cnn_ens = Model(inputs=input_holder,outputs=ave)

# Let's train the model 
from keras import backend as K
import functools
from itertools import product
import tensorflow as tf
# weights=np.array([[1,0],[-1,1]])
def w_categorical_crossentropy(y_true, y_pred):
    weights=np.array([[0.1,-0.1],[-0.1,0.1]])
    nb_cl = len(weights)
    final_mask = K.zeros_like(y_pred[:, 0])
    y_pred_max = K.max(y_pred, axis=1)
    y_pred_max = K.expand_dims(y_pred_max, 1)
    y_pred_max_mat = K.equal(y_pred, y_pred_max)
    for c_p, c_t in product(range(nb_cl), range(nb_cl)):

        final_mask += (K.cast(weights[c_t, c_p],K.floatx()) * K.cast(y_pred_max_mat[:, c_p] ,K.floatx())* K.cast(y_true[:, c_t],K.floatx()))
    return K.categorical_crossentropy(y_pred, y_true) * final_mask

cnn_ens.compile(loss=w_categorical_crossentropy, # 'categorical_crossentropy' 'mean_squared_error'
                optimizer='rmsprop', # 'adadelta' 'rmsprop'
                metrics=['accuracy'])

# the flow method yields batches of images indefinitely, with the given transofmrations
cnn_ens.fit_generator(datagen.flow(X_sub_train, y_sub_train_ohe, batch_size=128), 
                      steps_per_epoch=int(len(X_sub_train)/128), # how many generators to go through per epoch
                      epochs=2, verbose=1,
                      validation_data=(X_sub_test,y_sub_test_ohe),
                      callbacks=[EarlyStopping(monitor='val_loss', patience=4)]
                     )

Epoch 1/2
 61/117 [==============>...............] - ETA: 18s - loss: -0.9697 - acc: 0.3381

KeyboardInterrupt: 

In [24]:
from sklearn import metrics as mt
yhat_cnn = np.argmax(cnn_ens.predict(X_sub_test), axis=1)
rec_cnn = mt.f1_score(y_sub_test,yhat_cnn, average='macro')
acc_cnn = mt.accuracy_score(y_sub_test,yhat_cnn)
print("Recall: ", rec_cnn)
print("Accuracy: ", acc_cnn)

Recall:  0.417006274264
Accuracy:  0.474


In [ ]:
for a in yhat_cnn:
    print(a)

In [ ]:
from keras.callbacks import EarlyStopping
from keras.regularizers import l2 
l2_lambda = 0.0001
img_wh = 32

# Use Kaiming He to regularize ReLU layers: https://arxiv.org/pdf/1502.01852.pdf
# Use Glorot/Bengio for linear/sigmoid/softmax: http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf 
cnn = Sequential()

cnn.add(Conv2D(filters=32,
               input_shape = (img_wh,img_wh,1),
               kernel_size=(3,3),
               kernel_initializer='he_uniform', 
               kernel_regularizer=l2(l2_lambda),
               padding='same', 
               activation='relu')) # more compact syntax
cnn.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

cnn.add(Conv2D(filters=32,
               kernel_size=(3,3),
               kernel_initializer='he_uniform', 
               kernel_regularizer=l2(l2_lambda),
               padding='same', 
               activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))
    

# add one layer on flattened output
cnn.add(Dropout(0.25)) # add some dropout for regularization after conv layers
cnn.add(Flatten())
cnn.add(Dense(128, 
              activation='relu',
              kernel_initializer='he_uniform',
              kernel_regularizer=l2(l2_lambda)
       ))
cnn.add(Dropout(0.5)) # add some dropout for regularization, again!
cnn.add(Dense(NUM_CLASSES, 
              activation='softmax', 
              kernel_initializer='glorot_uniform',
              kernel_regularizer=l2(l2_lambda)
             ))

# Let's train the model 
cnn.compile(loss='categorical_crossentropy', # 'categorical_crossentropy' 'mean_squared_error'
              optimizer='rmsprop', # 'adadelta' 'rmsprop'
              metrics=['accuracy'])

# the flow method yields batches of images indefinitely, with the given transofmrations
cnn.fit_generator(datagen.flow(X_subtrain, y_train_ohe, batch_size=128), 
                  steps_per_epoch=int(len(X_train)/128), # how many generators to go through per epoch
                  epochs=50, verbose=1,
                  validation_data=(X_sub_test,y_test_ohe),
                  callbacks=[EarlyStopping(monitor='val_loss', patience=2)]
                 )

# Locus

### BC for Recall Score Evaluation
Recall scores penalizes the model for predicting false negatives. Our data encoding sets cars/trucks to be equal to 0 (negative class) and deer to be 1 (positive class). Thus, the recall metric scores the model pooly if it clasifies a deer (1) as a car (0). Since the business exists with the goal of reducing automobile collisions with deer, it is important that we do not assure mororists that there are no deer (only cars) on the road ahead, when in fact there are deer on the road.

### BC for binary_crossentropy
We are using binary_crossentropy as our loss function. Binary classification is the only binary classification metric available to us by Keras. We use this instead of something like mean squared error because this is a classification problem, not a regression one. 

Ideally, we would write a custom weighted binary crossentropy function, so that the model is only slightly penalized for producing false positives (see case for Recall Score Evaluation), but heavily penalized for producing false negatives. We tried [this implementation](https://github.com/fchollet/keras/issues/2115) from GitHib, but it seems to cause a vanishing gradient.


def weighted_binary_crossentropy(y_true, y_pred):
    return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)